### Selecting and preparing TIFF stacks for the website

In [ ]:
import os
import re
import sys
import glob
import json
import nrrd
import cv2
import vispy
import tifffile
import skimage
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import ndimage
from matplotlib import pyplot as plt
from matplotlib import colors as mplcolors

from skimage import feature
from skimage import morphology

In [ ]:
# HACK: path to local image-utils package (which is currently only on box)
sys.path.append('/Users/keith.cheveralls/Box/box-projects/image-utils/')
from image_utils import viz, utils

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# targets for demo
target_names = ['ATL2', 'ATL3', 'LAMP1', 'MTOR', 'POLR1C', 'RAB14', 'SEC13', 'TAF12', 'VAPA', 'CLTA']

### Generating z-projections to quickly select 'good' FOVs for each target

In [ ]:
def process_directory(root_dirpath_in):
    '''
    Generate projections of all TIFFs in all subdirectories of the root directory
    
    *** This method assumes that the TIFF stack dimension order is (z, channel, x, y) ***
    '''
    
    # drop trailing slashes
    root_dirpath_in, _ = os.path.split(root_dirpath_in)    
    root_dirpath_out = '%s_PROC' % root_dirpath_in
    
    for dirpath_in, subdirs, filenames in os.walk(root_dirpath_in):
        dirpath_out = dirpath_in.replace(root_dirpath_in, root_dirpath_out)
    
        for filename in filenames:
            ext = '.%s' % filename.split('.')[-1]
            if ext!='.tif':
                continue
            
            # HACK: only process FOVs corresponding to the targets 
            # we plan to use for the open-cell demo
            of_interest = False
            for name in target_names:
                if name in filename:
                    of_interest = True
                    
            if not of_interest:
                continue
            
            print('Processing %s' % filename)
            os.makedirs(dirpath_out, exist_ok=True)
            stack = utils.load(os.path.join(dirpath_in, filename))
            
            # make DAPI grayscale projection
            filepath = os.path.join(dirpath_out, filename.replace(ext, '_DAPI.tif'))
            if not os.path.isfile(filepath):
                im_dapi = utils.autogain(stack[:, 0, :, :].max(axis=0))
                tifffile.imsave(filepath, im_dapi)
                
            # make an RGB projection (DAPI in gray, GFP in green)
            filepath = os.path.join(dirpath_out, filename.replace(ext, '_RGB.tif'))
            if not os.path.isfile(filepath):
                proj_rgb = viz.make_rgb(imb=stack[:, 0, :, :], im_bg=stack[:, 1, :, :])
                tifffile.imsave(filepath, proj_rgb)

In [ ]:
process_directory('/Volumes/keith-external/Plate_Microscopy/mNG96wp3_thawed/')

In [ ]:
process_directory('/Users/keith.cheveralls/image-data/pipeline/mNG96wp1_Thawed/')

### Writing stacks as NRRD files

In [ ]:
# manually selected FOVs for each target

p1 = '/Volumes/keith-external/Plate_Microscopy/mNG96wp1_Thawed/'
p3 = '/Volumes/keith-external/Plate_Microscopy/mNG96wp3_Thawed/'
p1_clones = '/Volumes/keith-external/Plate_Microscopy/mNG96wp1_Clones/'

fovs_for_demo = {
    
    # plate1 clones
    'ATL2': os.path.join(p1_clones, 'ML0156_20190703_IJClean/A1-4_8_ATL2_PyProcessed_IJClean.tif'),
    'ATL3': os.path.join(p1_clones, 'ML0156_20190703_IJClean/A2-1_5_ATL3_PyProcessed_IJClean.tif'), 
    
    # plate1 thawed
    'MTOR': os.path.join(p1, 'ML0147_20190621_IJClean/D7_9_MTOR_PyProcessed_IJClean.tif'), 
    'VAPA': os.path.join(p1, 'ML0147_20190621_IJClean/H11_11_VAPA_PyProcessed_IJClean.tif'), 
    'RAB14': os.path.join(p1, 'ML0155_20190627_IJClean/E7_19_RAB14_PyProcessed_IJClean.tif'),
    'CLTA': os.path.join(p1, 'ML0143_20190612_IJClean/C12_6_CLTA_PyProcessed_IJClean.tif'),
    
    # plate3 thawed
    'POLR1C': os.path.join(p3, 'ML0158_20190709_IJClean/A3_1_POLR1C_PyProcessed_IJClean.tif'), 
    'SEC13': os.path.join(p3, 'ML0158_20190709_IJClean/F3_1_SEC13_PyProcessed_IJClean.tif'),
    'TAF12': os.path.join(p3, 'ML0164_20190718_IJClean/B1_17_TAF12_PyProcessed_IJClean.tif'), 
    'LAMP1': os.path.join(p3, 'ML0163_20190716_IJClean/E1_10_LAMP1_PyProcessed_IJClean.tif'),
    
}

In [ ]:
im = utils.load(fovs_for_demo['ATL2'])

In [ ]:
tifffile.imwrite('/Users/keith.cheveralls/test.tif', im[:, :, :256, :256])

In [ ]:
# directory in which to save the NRRD files
nrrd_dir = '../static/demo-data/stacks/'

In [ ]:
def to_nrrd(filepath_in, dirpath_out, write_files=False):
    '''
    Save a pipeline plate microscopy TIFF stack as a set of NRRD files (one for each channel)
    *** assumes order of dimensions is (z, channel, x, y)
    '''

    # assume filetype is 'tif'
    fileroot = filepath_in.split(os.sep)[-1].replace('.tif', '')

    im = utils.load(filepath_in)

    # crop one-quarter of the whole FOV
    im = im[:, :, :512, :512]
    
    # create an NRRD file for each channel
    for ind in (0, 1):
        filename = '%s_C%d.nrrd' % (fileroot, ind)
        filepath = os.path.join(dirpath_out, filename)
        
        if not write_files:
            continue
            
        if os.path.isfile(filepath):
            continue

        # move the z-dimension from the first to the last dimension
        im_out = np.moveaxis(utils.autogain(im[:, ind, :, :]), 0, -1)
        nrrd.write(filepath, im_out)
        
    return fileroot

In [ ]:
to_nrrd(fovs_for_demo['ATL2'], '')

In [ ]:
# write the NRRD files and log the fileroots
all_nrrd_filenames = {}
for target_name in fovs_for_demo.keys():
    all_nrrd_filenames[target_name] = to_nrrd(fovs_for_demo[target_name], nrrd_dir, write_files=True)

In [ ]:
all_nrrd_filenames

In [ ]:
with open('../src/demo-data/nrrd_filepaths.json', 'w') as file:
    json.dump(all_nrrd_filenames, file)

In [ ]:
# debugging: read an NRRD file
data, header = nrrd.read(filepath_nrrd)
im_nrrd.shape, data.shape, im_nrrd.dtype, data.dtype

### Aside: inspecting a single raw stack

In [ ]:
im = utils.load(filepath)

In [ ]:
# a single z-slice
viz.imshow(im[20, 1, :, :])

In [ ]:
# RGB z-projection (DAPI in gray)
viz.imshow(viz.make_rgb(img=im[:, 1, :, :], im_bg=im[:, 0, :, :]))

In [ ]:
# a single x-z plane
viz.imshow(viz.make_rgb(img=im[:, 1, 0, :], im_bg=im[:, 0, 0, :], gamma=.7))

### Aside: writing the stack as a directory of PNG slices

In [ ]:
fileroot = filepath.split(os.sep)[-1].split('.')[0]
png_dir = os.path.join('%s_PNG' % root, fileroot)
os.makedirs(png_dir, exist_ok=True)
fileroot

In [ ]:
# channels
for channel_ind in range(im.shape[1]):
    im_out = utils.autogain(im[:, channel_ind, :, :])
    
    # z-slices
    for z_ind in range(im.shape[0]):
        s = im_out[z_ind, :, :]
        filename = '%s_C%d_Z%02d.png' % (fileroot, channel_ind, z_ind)
        cv2.imwrite(os.path.join(png_dir, filename), s)